# Watched Lists

---

keeping track of element changes in a list

In [1]:
from spectate import Spectator, WatchedType

**`ListSpectator`** inherits from `Spectator` and implements the callbacks `instance_will_call` and `instance_post_call`. This spectator understands what will happen when `list.__setitem__` is called and overrides the preemptive callback `instance_will_call`, which captures the arguments of `__setitem__`, to determine whether an element of the list will change and caches the old and new values if they're different. The overriden callback `instance_post_call` then checks for a change, and prints it if there was.

In [2]:
class ListSpectator(Spectator):
    
    def __init__(self, base, inst):
        super(ListSpectator, self).__init__(base, inst)
        self._change = {'old': None, 'new': None}
    
    def instance_will_call(self, name, args, kwargs):
        # we know the name will be __setitem__
        # since that's all we're spectating
        index, new = args
        try:
            old = self.inst[index]
        except:
            pass
        else:
            if old != new:
                self._change['old'] = {index: old}
                self._change['new'] = {index: new}
    
    def instance_post_call(self, name, returned):
        o, n = self._change['old'], self._change['new']
        if o and n:
            print("%s : %r -> %r" % (name, o, n))
            self._change['old'] = None
            self._change['new'] = None

**`WatchedType`** is used to create a `WatchedList` class that assigns a `ListSpectator` to each of its instances.

In [3]:
WatchedList = WatchedType('WatchedList', list, ListSpectator, '__setitem__')

The instances of `WatchedList` will behave exactly like a `list` in every way. The only difference being that when a user decides to change the value of a preexisting element (thus envoking `__setitem__`), the instance's spectator is notified, and will print that change once the action is complete.

In [4]:
wl = WatchedList([1, 2, 3])
sp = wl.instance_spectator

msg = "spectator is linked to instance: %s"
print(msg % (sp.inst is wl))

spectator is linked to instance: True


In [5]:
# does not notify because
# values are the same
wl[0] = 1

In [6]:
# will notify when the
# values are different
wl[0] = 0

__setitem__ : {0: 1} -> {0: 0}
